# Runnables

# The Why

    - In the early langchain days each components in langchain like models, prompts, retrivers, parsers etc.. all had different interface.
    - This was because when chatGpt and other LLMs provided APIs they all had different schema for interacting with it, so the langchain was focused on making common interface between different providers.
    - Due to which each component they made had different schema within themselves.
    - Also the langchain team noticed that some components are used togther every times like prompts and llm, so instead of using prompts and llm seperately langchain provided a component called LLMChain with which it made easy for using llm and prompt with one single function.
    - Likewise Langchain started providing such chain components like RetriverQA, SequentialChain, RouterChain, AgentExecutorChain, ConversationalRetrievalChain etc..

![alt text](./Images/chainComponents.png)


    - They started creating chain for tasks they are getting reused, so that instead of using separate components, user can use chains.
    - The problem arised when, chains started getting popularity and there were so much chains that made langchain code base large and difficult to maintain.
    - Since the components like models prompts are not standardised, the langchain team was forced to create such huge no.of chains.
    - By standardised means, for using prompt component the user have to call prompt.format(), for using model we have to call llm.predict(), likewise each component had their own methods for using it.
    - That's why they started creating custom functions and chains so that the user have not worry about this problem. But this was not a good practise.
    - Since these components was not build to connect together and use, langchain has to write manual code for making it compatible.
    - So the langchain started to standardise the components with the concept of **runnables**.


In [1]:
import random

class NakliLLM:

  def __init__(self):
    print('LLM created')

  def predict(self, prompt):

    response_list = [
        'Delhi is the capital of India',
        'IPL is a cricket league',
        'AI stands for Artificial Intelligence'
    ]

    return {'response': random.choice(response_list)}

In [2]:
class NakliPromptTemplate:

  def __init__(self, template, input_variables):
    self.template = template
    self.input_variables = input_variables

  def format(self, input_dict):
    return self.template.format(**input_dict)

In [3]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [4]:
prompt = template.format({'length':'short','topic':'india'})

In [5]:
llm = NakliLLM()

LLM created


In [6]:
llm.predict(prompt)

{'response': 'IPL is a cricket league'}

In [7]:
class NakliLLMChain:

  def __init__(self, llm, prompt):
    self.llm = llm
    self.prompt = prompt

  def run(self, input_dict):

    final_prompt = self.prompt.format(input_dict)
    result = self.llm.predict(final_prompt)

    return result['response']


In [8]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [9]:
llm = NakliLLM()

LLM created


In [10]:
chain = NakliLLMChain(llm, template)

In [11]:
chain.run({'length':'short', 'topic': 'india'})

'AI stands for Artificial Intelligence'

# What

![alt text](./Images/runnables.png)

- We can think of runnables as a unit of work
    - Each runnables will take a input, process it, and can provide a output
- Every runnable in langchain have a common interface:
    - invoke(), batch(), stream() method for calling it
    - Also we can connect each runnables and it will form another, think of it like a lego block

# How

- For creating common interface they used concept **abstract class and methods**.

In [12]:
from abc import ABC, abstractmethod

class Runnables(ABC):
    
    @abstractmethod
    def invoke(input_data):
        pass
    

In [13]:
class DummyLLM(Runnables):
    def __init__(self):
        print("LLM Created")
        
    def predict(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]

        return {'response': random.choice(response_list)}

llm = DummyLLM()

TypeError: Can't instantiate abstract class DummyLLM without an implementation for abstract method 'invoke'

In [17]:
class DummyLLM(Runnables):
    def __init__(self):
        print("LLM Created")
    
    # Abstract Method
    def invoke(self, prompt):
        
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]

        return {'response': random.choice(response_list)}
    
    def predict(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]

        return {'response': random.choice(response_list)}
    
llm = DummyLLM()
    

LLM Created


In [19]:
class NakliPromptTemplate(Runnables):
    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self,input_dict):
        return self.template.format(**input_dict)
    
    def format(self, input_dict):
        return self.template.format(**input_dict)
    
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [40]:
class DummyStrOutputParser(Runnables):
    def __init__(self):
        pass
    def invoke(self,input_data):
        return input_data['response']

In [41]:
class RunnableConnector(Runnables):
    def __init__(self,runnable_list):
        self.runnable_list = runnable_list
    
    def invoke(self, input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
        return input_data

In [42]:
parser = DummyStrOutputParser()

In [43]:
chain = RunnableConnector([template,llm,parser])

In [44]:
chain.invoke({'length':'short', 'topic': 'india'})

'AI stands for Artificial Intelligence'

### Connecting Runnables

In [45]:
template1 = NakliPromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

In [46]:
template2 = NakliPromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

In [47]:
chain1 = RunnableConnector([template1, llm])

In [48]:
chain2 = RunnableConnector([template2, llm, parser])

In [49]:
final_chain = RunnableConnector([chain1, chain2])

In [50]:
final_chain.invoke({'topic':'cricket'})

'Delhi is the capital of India'

# Langchain Runnables

![alt text](./Images/runnables_types.png)

In [51]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="qwen2.5-coder:7b",
    model_provider="ollama",
    temperature = 0.0
)

# Runnable Passthrough

- RunnablePassthroughis a special Runnable primitive that simply returns the input as output without modifying it.

In [52]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

{'joke': 'Why did the cricket player break up with his girlfriend?\n\nBecause he found out she was a bit of a no-show!', 'explanation': 'The joke is a play on words and cultural references. Cricket is a popular sport in many countries, particularly in England where this joke originates. The term "no-show" has two meanings: one is when someone doesn\'t show up for an event or appointment, and the other is a slang term used to describe a woman who is sexually uninterested or unavailable.\n\nIn the context of the joke, the cricket player\'s girlfriend breaking up with him because she was a "bit of a no-show" could be interpreted as her being unreliable or not showing up when expected. However, it could also be interpreted as her being sexually uninterested or unavailable, which is a play on the double meaning of "no-show."\n\nThe punchline is unexpected and clever because it plays on the double meaning of "no-show," making the joke both funny and culturally relevant.'}


# Runnable Sequence

- RunnableSequence is a sequential chain of runnables in LangChain that executes each step one after another, passing the output of one step as the input to the next. 
- It is useful when you need to compose multiple runnables together in a structured workflow.

In [53]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)

print(chain.invoke({'topic':'AI'}))

The joke is a play on words. "AI" stands for Artificial Intelligence, but it can also be read as "A I," which sounds like "I am." The punchline "because it had a splitting headache!" is a reference to the common phrase "splitting headache," which means a very severe or intense headache. So, the joke is saying that the AI (which is actually just a computer program) went to therapy because it was experiencing a very severe headache.


# Runnable Lambda

- RunnableLambdais a runnable primitive that allows you to apply custom Python functions within an AI pipeline. 
- It acts as a middlewarebetween different AI components, enabling preprocessing, transformation, API calls, filtering, and post-processingin a LangChain workflow.

In [54]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Why did the AI go to therapy?

Because it had a splitting headache! 
 word count - 13


# Runnable Branch

- RunnableBranchis a control flow componentin LangChain that allows you to conditionally route input data to different chains or runnablesbased on custom logic. 
- It functions like an if/elif/elseblock for chains —where you define a set of condition functions, each associated with a runnable (e.g., LLM call, prompt chain, or tool). The first matching condition is executed. If no condition matches, a default runnableis used (if provided).

In [55]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))




### Summary of the Report: Russia-Ukraine Conflict

The Russia-Ukraine conflict is one of the most significant geopolitical events in recent years. The roots of this conflict can be traced back to historical, cultural, economic, and political factors. This report aims to provide a comprehensive overview of the conflict, including its history, key players, major events, and potential outcomes.

#### Historical Context
The relationship between Russia and Ukraine has been complex for centuries. Historically, both countries have shared a rich cultural heritage and linguistic ties. However, the modern state of Ukraine emerged in the late 19th century as part of the Russian Empire. Following the collapse of the Soviet Union in 1991, Ukraine gained independence but faced numerous challenges.

#### Key Players
- **Russia**: The primary actor is the Russian Federation, led by President Vladimir Putin.
- **Ukraine**: An independent nation with its own government, military, and political system.


# Runnable Parallel
- RunnableParallel is a runnable primitive that allows multiple runnables  to execute in parallel. 
- Each runnable receives the same input and processes it independently, producing a dictionary of outputs.

In [56]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)


parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'tweet': RunnableSequence(prompt1, model, parser),
    'linkedin': RunnableSequence(prompt2, model, parser)
})

result = parallel_chain.invoke({'topic':'AI'})

print(result['tweet'])
print(result['linkedin'])


"Exciting times ahead as artificial intelligence continues to revolutionize our world! From healthcare to transportation, AI is making incredible strides in improving efficiency and enhancing human capabilities. #AI #Innovation #FutureIsNow"
🚀 Exciting Times Ahead! 🌟

As we step into 2024, the world of Artificial Intelligence (AI) is on the cusp of revolutionizing industries and enhancing lives in ways we can only begin to imagine. From healthcare diagnostics to sustainable energy solutions, AI's impact is becoming increasingly evident.

🤖 How AI is Shaping Our Future:
- **Healthcare**: AI is helping doctors diagnose diseases more accurately and quickly, improving patient outcomes.
- **Finance**: It's revolutionizing how banks operate, from fraud detection to personalized financial advice.
- **Transportation**: Self-driving cars are not just a dream; they're closer than ever, promising safer roads and reduced traffic congestion.
- **Education**: AI is personalizing learning experiences